<a href="https://colab.research.google.com/github/EsraaMosaad/Practice_NLP/blob/main/text_clssification_spamORharm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ***practice*** **Using BERT and Tensorflow 2.0, we will write simple code to classify emails as spam or not spam. BERT will be used to generate sentence encoding for all emails and after that we will use a simple neural network with one drop out layer and one output layer.I've been using this YouTube Tutorial Channel https://www.youtube.com/watch?v=hOCDJyZ6quA**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')



In [ ]:
!cp /content/drive/MyDrive/kaggel_api/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d arunasivapragasam/spam-or-ham
! unzip /content/spam-or-ham.zip

  0% 0.00/209k [00:00<?, ?B/s]
100% 209k/209k [00:00<00:00, 90.1MB/s]
Archive:  /content/spam-or-ham.zip
  inflating: SMSCollection.csv       


In [ ]:
import pandas as pd
import tensorflow as tf

# **data preprossing** spam & ham sms

In [ ]:
df=pd.read_csv("/content/SMSCollection.csv")

In [ ]:
df

,Class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df.groupby('Class').describe()

sms                                                               
      count unique                                                top freq
Class                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [ ]:
df['Class'].value_counts()

ham     4825
spam     747
Name: Class, dtype: int64

Balance data

In [ ]:
df_spam=df[df['Class']=='spam']
df_spam.shape

(747, 2)

In [ ]:
df_ham=df[df['Class']=='ham']
df_ham.shape

(4825, 2)

In [ ]:
df_resampling=df_ham.sample(df_spam.shape[0])
df_resampling.shape

(747, 2)

In [ ]:
balance_df=pd.concat([df_resampling,df_spam])
balance_df.shape

(1494, 2)

In [ ]:
balance_df.groupby(["Class"]).describe()

sms                                                               
      count unique                                                top freq
Class                                                                     
ham     747    732                             Sorry, I'll call later    8
spam    747    653  Please call our customer service representativ...    4

convert spam to numeric by adding nwe colum to data frame 

In [ ]:
balance_df['spam']=balance_df['Class'].apply(lambda x: 1 if x=='spam' else 0)
balance_df.head(10)

,Class,sms,spam
2560,ham,I also thk too fast... Xy suggest one not me. ...,0
69,ham,I plane to give on this month end.,0
4714,ham,Big brother‘s really scraped the barrel with t...,0
4269,ham,Hey so whats the plan this sat?,0
3410,ham,Whats that coming over the hill..... Is it a m...,0
1421,ham,No. She's currently in scotland for that.,0
832,ham,Hi mate its RV did u hav a nice hol just a mes...,0
1558,ham,Wat r u doing?,0
1001,ham,No..but heard abt tat..,0
1552,ham,In e msg jus now. U said thanks for gift.,0


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(balance_df['sms'], balance_df['spam'],stratify=balance_df['spam'])


In [ ]:
x_train.head(4)

5       FreeMsg Hey there darling it's been 3 week's n...
3463    Bloomberg -Message center +447797706009 Why wa...
2095    PRIVATE! Your 2004 Account Statement for 07742...
2676    I'm sick !! I'm needy !! I want you !! *pouts*...
Name: sms, dtype: object

# **BERT_MODEL**

In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 33.0 MB/s 
     |████████████████████████████████| 497.9 MB 37 kB/s 
     |████████████████████████████████| 5.8 MB 55.5 MB/s 
     |████████████████████████████████| 1.4 MB 61.9 MB/s 
     |████████████████████████████████| 462 kB 74.1 MB/s 


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [ ]:
BERT_preprocess_model=hub.KerasLayer(preprocess_url)
BERT_encoder=hub.KerasLayer(encoder_url)

In [ ]:
def get_sentance_embeding(x):
  text_preprocessed=BERT_preprocess_model(x)
  BERT_result=BERT_encoder(text_preprocessed)
  print(len(BERT_result))
  return BERT_result['pooled_output']

In [ ]:
embedding=get_sentance_embeding(['banana',"orange",'mango','bill gates','elon musk'])

4


In [ ]:
embedding[0].shape

TensorShape([768])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([embedding[3]],[embedding[4]])

array([[0.9164156]], dtype=float32)

In [ ]:
#BERT layer
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string ,name="text")
preprocessed_text=BERT_preprocess_model(text_input)
outputs=BERT_encoder(preprocessed_text)
#Neural network layers

l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)
# construct final model
model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
35/35 [==============================] - 20s 335ms/step - loss: 0.5985 - accuracy: 0.7143 - precision: 0.7027 - recall: 0.7429
Epoch 2/10
35/35 [==============================] - 12s 347ms/step - loss: 0.4893 - accuracy: 0.8304 - precision: 0.8063 - recall: 0.8696
Epoch 3/10
35/35 [==============================] - 12s 356ms/step - loss: 0.4290 - accuracy: 0.8545 - precision: 0.8428 - recall: 0.8714
Epoch 4/10
35/35 [==============================] - 13s 362ms/step - loss: 0.3835 - accuracy: 0.8821 - precision: 0.8702 - recall: 0.8982
Epoch 5/10
35/35 [==============================] - 12s 347ms/step - loss: 0.3541 - accuracy: 0.8893 - precision: 0.8733 - recall: 0.9107
Epoch 6/10
35/35 [==============================] - 12s 341ms/step - loss: 0.3289 - accuracy: 0.9036 - precision: 0.8857 - recall: 0.9268
Epoch 7/10
35/35 [==============================] - 12s 340ms/step - loss: 0.3222 - accuracy: 0.8938 - precision: 0.8848 - recall: 0.9054
Epoch 8/10
35/35 [================

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_pred=model.predict(x_test)
y_pred=y_pred.flatten()

In [ ]:
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from  sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,y_pred)
cm

In [ ]:
from matplotlib import pyplot as plt 
import seaborn as sn
sn.heatmap(cm,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')